In [18]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [19]:
from google import genai
client = genai.Client(api_key="****************)
model = "gemini-2.5-flash"

In [20]:
def   gemini_response(question):
    response = client.models.generate_content(
        model=model,
        contents=question,
    )
    return response.text

In [14]:
reply = gemini_response("can u give me an image of a little boy?")
print(reply)

As a text-based AI, I can't directly display images here.

However, I can *describe* one for you in detail, or even give you a prompt you could use with an image-generating AI!

**Here's a description of a little boy:**

"Imagine a cheerful little boy, perhaps around 4 or 5 years old, with bright, curious brown eyes and a wide, dimpled smile. His sandy blonde hair is slightly tousled, perhaps from playing. He's wearing a bright blue t-shirt with a playful animal design and comfortable khaki shorts, his knees a little scuffed. He might be holding a small, colorful toy car or looking directly at you with an innocent, playful expression. The setting could be a softly blurred, sunny park in the background."

Would you like me to describe something more specific, or give you a prompt for an image AI?


In [16]:
pip install Pillow

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 1.2 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.1 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.1 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.1 MB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 883.2 kB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 883.2 kB/s eta 0:00:02
   --------------------------- ------------ 1.8/2.7 MB 868.0 kB/s eta 0:00:01
   --------------------------- ------------ 1.8/2.7 MB 868.0 kB/s eta 0:00:01
   ----------------------

In [21]:
###text to image###
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

contents = ('Hi, can you create a 3d rendered image of a pig '
            'with wings and a top hat flying over a happy '
            'futuristic scifi city with lots of greenery?')

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save('gemini-native-image.png')
    image.show()

I will create a 3D rendering of a whimsical scene: a pink pig with small, delicate white wings gracefully soars above a vibrant, futuristic cityscape. The pig will be wearing a stylish black top hat perched jauntily on its head. Below, the sci-fi city will feature sleek, modern buildings interspersed with lush green spaces, creating a cheerful and utopian atmosphere.




In [ ]:
###text to video###
import time
from google import genai
from google.genai import types

client = genai.Client()  # read API key from GOOGLE_API_KEY

operation = client.models.generate_videos(
    model="veo-2.0-generate-001",
    prompt="Panning wide shot of a calico kitten sleeping in the sunshine",
    config=types.GenerateVideosConfig(
        person_generation="dont_allow",  # "dont_allow" or "allow_adult"
        aspect_ratio="16:9",  # "16:9" or "9:16"
    ),
)

while not operation.done:
    time.sleep(20)
    operation = client.operations.get(operation)

for n, generated_video in enumerate(operation.response.generated_videos):
    client.files.download(file=generated_video.video)
    generated_video.video.save(f"video{n}.mp4")  # save the video


In [23]:
###text to audio###
from google.genai import types
import wave

def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

response = client.models.generate_content(
   model="gemini-2.5-flash-preview-tts",
   contents="Say cheerfully: Have a wonderful day!",
   config=types.GenerateContentConfig(
      response_modalities=["AUDIO"],
      speech_config=types.SpeechConfig(
         voice_config=types.VoiceConfig(
            prebuilt_voice_config=types.PrebuiltVoiceConfig(
               voice_name='Kore',
            )
         )
      ),
   )
)

data = response.candidates[0].content.parts[0].inline_data.data

file_name='out.wav'
wave_file(file_name, data) # Saves the file to current directory

In [24]:
###url understanding###
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

model_id = "gemini-2.5-flash"

url_context_tool = Tool(
    url_context = types.UrlContext
)

response = client.models.generate_content(
    model=model_id,
    contents="Compare recipes from https://www.loveandlemons.com/how-to-make-hard-boiled-eggs/ and https://www.allrecipes.com/recipe/46822/indian-chicken-curry-ii/",
    config=GenerateContentConfig(
        tools=[url_context_tool],
        response_modalities=["TEXT"],
    )
)

for each in response.candidates[0].content.parts:
    print(each.text)
# get URLs retrieved for context
print(response.candidates[0].url_context_metadata)


The first recipe, "How to Make Hard Boiled Eggs," from Love and Lemons, focuses on a basic cooking technique to achieve perfectly hard-boiled eggs with vibrant yellow yolks that are easy to peel. The method involves placing eggs in a pot, covering them with cold water, bringing to a rolling boil, then turning off the heat, covering, and letting them sit for 10-12 minutes. A crucial step for easy peeling and to stop the cooking process is transferring the eggs to an ice bath for at least 14 minutes. The recipe also offers tips like using older eggs for easier peeling and storing raw eggs upside down for centered yolks.

In contrast, the "Indian Chicken Curry" recipe from Allrecipes is for a flavorful main dish featuring chicken stewed in a creamy, spiced, tomato-based sauce. This recipe calls for ingredients such as olive oil, onion, garlic, curry powder, cinnamon, paprika, bay leaf, ginger, sugar, salt, chicken breast, tomato paste, plain yogurt, coconut milk, lemon juice, and cayenne 

In [28]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [38]:
### google search ###
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

model_id = "gemini-2.5-flash"

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response = client.models.generate_content(
    model=model_id,
    contents="When is the next total solar eclipse in switzerland?",
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

for each in response.candidates[0].content.parts:
    print(each.text)
# Example response:
# The next total solar eclipse visible in the contiguous United States will be on ...

# To get grounding metadata as web content.
print(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

The next total solar eclipse visible in Switzerland will occur on September 3, 2081. While partial solar eclipses will be visible in Switzerland before this date, such as on August 12, 2026, and August 2, 2027, the path of totality for these eclipses will cross other parts of Europe, including Spain and Iceland.
<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);